In [1]:
import numpy as np
import pandas as pd

In [3]:
df1=pd.read_csv("../r2Data/user_vendor_series_data.csv",usecols=['user_id','business_id','stars_x','text','city','latitude','longitude','review_count','categories'],nrows=100000,keep_default_na=False)

## Buliding basic collaborative model

In [15]:
df1.drop(columns=['address','useful','funny','cool'],inplace=True)

In [33]:
df2=df1.iloc[:20000,:]

In [5]:
df1.head(3)

,user_id,business_id,stars_x,text,city,latitude,longitude,review_count,categories
0,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,"If you decide to eat here, just be aware it is...",North Wales,40.210196,-75.223639,169,"Restaurants, Breakfast & Brunch, Food, Juice B..."
1,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5,I've taken a lot of spin classes over the year...,Philadelphia,39.952103,-75.172753,144,"Active Life, Cycling Classes, Trainers, Gyms, ..."
2,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3,Family diner. Had the buffet. Eclectic assortm...,Tucson,32.207233,-110.980864,47,"Restaurants, Breakfast & Brunch"


In [7]:
pivot_table_df=pd.pivot_table(df1,index=['business_id'],columns=['user_id'],values='stars_x')

In [8]:
pivot_table_df.fillna(value=0,inplace=True)

In [9]:
pivot_table_df

user_id,---2PmXbF47D870stH1jqA,--4AjktZiHowEIBCMd4CZA,--E0uVPphTORm_OiZ5KCvA,--KMTwCrhKKUmr7riuS4WQ,--S8M395r8NtOCvS2LRfDw,--UizzbnQlZg7bEv2oXEyg,--WhhlIim6ISY5yhFYZreg,--_H9j6ggxvqhh9nPofZwg,--_r6E98SNIrGU7weyNxbw,--ccVMj2PN6Z9qtdOdlung,...,zzeRzizkihWHz9bVAvbcVw,zzgLgszu48af4NJULymzAA,zzgMuJgxmToqcJ5iu1TngQ,zziJLt25YU6dp01sewR-IQ,zzjThJ4A1m1N78gquSR_QA,zzljYL9c22HWRO3VvJ1_Pg,zzsPf8xNW11nd0B6MZqfRw,zzsqjDvanJhH9tn8NautOQ,zztkCqqgR6VntYbqio4UTQ,zzxUT-fFNXpp5i5xJHmvpA
business_id,,,,,,,,,,,,,,,,,,,,,
--ZVrH2X2QXBFdCilbirsw,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
--_9CAxgfXZmoFdNIRrhHA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-02xFuruu85XmDn2xiynJw,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-0Ym1Wg3bXd_TDz8JtvOQg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-0fvhILrC9UsQ6gLNpZlTQ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zyKTy79BwgCLdUGBiuYPxA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zzIF9qp2UoHN48EeZH_IDg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zzKVqizyl3QCb3GZmg_cNg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
from scipy.sparse import csr_matrix

In [11]:
df_sparse=csr_matrix(pivot_table_df)

In [12]:
from sklearn.neighbors import NearestNeighbors
model1=NearestNeighbors(algorithm='brute')

In [13]:
model1.fit(df_sparse)

NearestNeighbors(algorithm='brute')

In [44]:
distance,rec=model1.kneighbors(pivot_table_df.iloc[20,:].values.reshape(1,-1),n_neighbors=1000)

In [47]:
rec

array([[  20, 3597, 6917, 9183,  904, 5173, 8340, 9806, 8724, 5899, 6933,
        8212, 7910, 9418, 9713, 4062, 8250, 8248, 2639, 4797, 7988, 2717,
        9041, 6726, 7274, 7512, 8310, 4298, 9182, 9633, 2887, 7966, 9477,
        9934, 9764, 9300, 9414, 5642, 4585, 6973, 9229, 7756, 5672, 8460,
        8467, 8782, 6549, 9945, 6740, 5739, 5204, 9472, 9660, 8589, 8602,
        9842, 8634,  982, 9447, 5251, 8646, 7836, 4420, 6417, 6943, 7209,
        9379, 8830, 9403, 8092, 5942, 8119, 8115, 1308, 3410, 9938,  681,
        6041, 3404, 6074, 9270, 3444, 9690, 8139, 2494, 1693, 6958, 9430,
        9193,  979, 7319, 8879, 8161, 6205, 3797, 4440, 9386, 7072, 7069,
        4055, 8656, 6557, 9520, 5346, 9721, 4232, 3861, 4916, 6412, 9364,
        4644, 7737, 9274, 5421, 9846, 3487, 6728, 5395, 8643, 3794, 5396,
        7004, 7925, 8898, 3553, 5407, 8241, 2566, 3727, 8963, 8252, 2585,
        5018, 5416, 9726, 4296, 8834, 3790, 3567, 4022, 2631, 4091, 4789,
        9543, 3679, 2658, 8274, 2665, 

In [48]:
pivot_table_df.iloc[3353,:]

user_id
---2PmXbF47D870stH1jqA    0.0
--4AjktZiHowEIBCMd4CZA    0.0
--E0uVPphTORm_OiZ5KCvA    0.0
--KMTwCrhKKUmr7riuS4WQ    0.0
--S8M395r8NtOCvS2LRfDw    0.0
                         ... 
zzljYL9c22HWRO3VvJ1_Pg    0.0
zzsPf8xNW11nd0B6MZqfRw    0.0
zzsqjDvanJhH9tn8NautOQ    0.0
zztkCqqgR6VntYbqio4UTQ    0.0
zzxUT-fFNXpp5i5xJHmvpA    0.0
Name: KSTWGtRlYowMIuFspfvAFg, Length: 79345, dtype: float64

In [49]:
df1[df1['business_id']=='KSTWGtRlYowMIuFspfvAFg']

,user_id,business_id,stars_x,text,city,latitude,longitude,review_count,categories
8683,N1T5EKuyTSSCx_j49rTu_A,KSTWGtRlYowMIuFspfvAFg,1,Yelp wants you to give stars but this place is...,New Orleans,29.999599,-90.062288,17,"Restaurants, Pizza, Food, Italian, Fast Food, ..."


In [46]:
df1[df1['business_id']=='-5jrwZnndGs9q3akdkJYJA']

,user_id,business_id,stars_x,text,city,latitude,longitude,review_count,categories
4298,DgcCohYTe6UKkaCJc-kFDQ,-5jrwZnndGs9q3akdkJYJA,5,Went on Saturday night for the 9:45 show to se...,Edmonton,53.523326,-113.624791,52,"Comedy Clubs, Nightlife"
7456,wZYFf3deapERlQ3pu2aBhw,-5jrwZnndGs9q3akdkJYJA,4,I can't speak too much to the kind of talent t...,Edmonton,53.523326,-113.624791,52,"Comedy Clubs, Nightlife"
8052,hQgWwIjEbn47nN2BIr_LyA,-5jrwZnndGs9q3akdkJYJA,2,Since this is a live act venue you cant review...,Edmonton,53.523326,-113.624791,52,"Comedy Clubs, Nightlife"
9381,V45ZP0J5Fe0D5vWHRA8jng,-5jrwZnndGs9q3akdkJYJA,5,Always a fun time! We came to see Pete Correal...,Edmonton,53.523326,-113.624791,52,"Comedy Clubs, Nightlife"
20343,r8n6XIU1_-v1LDYZNPVf-w,-5jrwZnndGs9q3akdkJYJA,4,I really enjoyed the night out we had here for...,Edmonton,53.523326,-113.624791,52,"Comedy Clubs, Nightlife"
20648,1aqLfvwhitfOw2vJcp1wsQ,-5jrwZnndGs9q3akdkJYJA,4,My mom was in town visiting for the long weeke...,Edmonton,53.523326,-113.624791,52,"Comedy Clubs, Nightlife"
44075,Vc4QSYKAOf8NXriGXo919Q,-5jrwZnndGs9q3akdkJYJA,3,I went here the other night to celebrate a fri...,Edmonton,53.523326,-113.624791,52,"Comedy Clubs, Nightlife"
48407,zoA4G79atCGUpACqklty-Q,-5jrwZnndGs9q3akdkJYJA,3,If we had a choice of half stars this would be...,Edmonton,53.523326,-113.624791,52,"Comedy Clubs, Nightlife"
61233,_nMH2vC7cM47bStZ7dKqfQ,-5jrwZnndGs9q3akdkJYJA,5,A wonderful way to spend an evening with frien...,Edmonton,53.523326,-113.624791,52,"Comedy Clubs, Nightlife"
63041,zoQ7PP4MlKL_XLZKuIp2Zg,-5jrwZnndGs9q3akdkJYJA,4,The Comic Strip is the best comedy club in Edm...,Edmonton,53.523326,-113.624791,52,"Comedy Clubs, Nightlife"


In [45]:
pivot_table_df.iloc[20,:]

user_id
---2PmXbF47D870stH1jqA    0.0
--4AjktZiHowEIBCMd4CZA    0.0
--E0uVPphTORm_OiZ5KCvA    0.0
--KMTwCrhKKUmr7riuS4WQ    0.0
--S8M395r8NtOCvS2LRfDw    0.0
                         ... 
zzljYL9c22HWRO3VvJ1_Pg    0.0
zzsPf8xNW11nd0B6MZqfRw    0.0
zzsqjDvanJhH9tn8NautOQ    0.0
zztkCqqgR6VntYbqio4UTQ    0.0
zzxUT-fFNXpp5i5xJHmvpA    0.0
Name: -5jrwZnndGs9q3akdkJYJA, Length: 79345, dtype: float64